# Part 3: Unbiased Evaluation using a New Test Set

In this part, we are given a new test set (`/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`). We can now take advantage of the entire smart sample that we created in Part I. 

* Retrain a pipeline using the optimal parameters that the pipeline learned. We don't need to repeat GridSearch here. 

## Import modules as needed

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys
import itertools
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.covariance import EllipticEnvelope
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import joblib

## Load smart sample and the best pipeline from Part II

In [2]:
X1, Y1, final_model = joblib.load('pipeline3_model_RF.pkl')
X2, Y2, anamoly_model = joblib.load('model3_outlier.pkl')
dataset_train = joblib.load('preprocessed_data_part1.pkl')


##  Retrain a pipeline using the full sampled training data set

Use the full sampled training data set to train the pipeline.

In [3]:
# Add code below this comment  (Question #E301)
# ----------------------------------
X_train = np.array(dataset_train.iloc[:,:-1]) # Pull all rows, each column except the last
y_train = np.array(dataset_train.went_on_backorder) # Pull just the went_on_backorder column
print('X_train', X_train.shape, 'y_train', y_train.shape)

anamoly_model.fit(X_train, y_train)
outliers_train = anamoly_model.predict(X_train)==-1
print(f"Num of outliers = {np.sum(outliers_train)}")
X_train_inliers = X_train[~outliers_train]
y_train_inliers = y_train[~outliers_train]

final_model.fit(X_train_inliers, y_train_inliers)

print("Parameters of the best model are:",final_model.best_estimator_)
print("Score of the best model:", final_model.best_score_)


X_train (22586, 21) y_train (22586,)
Num of outliers = 1130


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Parameters of the best model are: Pipeline(steps=[('scale', StandardScaler()),
                ('RFE',
                 RFE(estimator=LogisticRegression(), n_features_to_select=20)),
                ('RandomForest',
                 RandomForestClassifier(criterion='entropy', min_samples_leaf=2,
                                        min_samples_split=6,
                                        n_estimators=30))])
Score of the best model: 0.9039431889237818


### Save the trained model with the pickle library.

In [4]:
# Add code below this comment  
# -----------------------------
joblib.dump([X_train_inliers, y_train_inliers, final_model], 'final_model.pkl')

['final_model.pkl']


## Load the Testing Data and evaluate your model

 * `/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`
 
* We need to preprocess this test data (**follow** the steps similar to Part I)
* **If you have fitted any normalizer/standardizer in Part 2, then we have to transform this test data using the fitted normalizer/standardizer!**

In [5]:
# Preprocess the given test set  (Question #E302)
# ----------------------------------

# Dataset location
DATASET = '/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)

dataset.head().transpose()


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4
sku,3424460,3285353,3489864,3287008,3496133
national_inv,52.0,551.0,11.0,13.0,4.0
lead_time,2.0,8.0,12.0,14.0,2.0
in_transit_qty,0.0,80.0,0.0,0.0,0.0
forecast_3_month,0.0,490.0,0.0,0.0,0.0
forecast_6_month,0.0,950.0,0.0,0.0,1.0
forecast_9_month,0.0,1460.0,0.0,0.0,2.0
sales_1_month,0.0,224.0,0.0,3.0,0.0
sales_3_month,0.0,737.0,0.0,3.0,1.0
sales_6_month,1.0,1338.0,1.0,5.0,8.0


# Preprocessing

In [6]:
dataset.shape

(242076, 23)

In [7]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
national_inv,242075.0,499.751028,29280.390793,-25414.0,4.00,15.00,81.00,12145792.0
lead_time,227351.0,7.923018,7.041410,0.0,4.00,8.00,9.00,52.0
in_transit_qty,242075.0,36.178213,898.673127,0.0,0.00,0.00,0.00,265272.0
forecast_3_month,242075.0,181.472345,5648.874620,0.0,0.00,0.00,4.00,1510592.0
forecast_6_month,242075.0,348.807304,10081.797119,0.0,0.00,0.00,12.00,2157024.0
forecast_9_month,242075.0,508.296301,14109.723787,0.0,0.00,0.00,20.00,3162260.0
sales_1_month,242075.0,51.478195,1544.678350,0.0,0.00,0.00,4.00,349620.0
sales_3_month,242075.0,172.139316,5164.243624,0.0,0.00,1.00,14.00,1099852.0
sales_6_month,242075.0,340.425414,9386.523492,0.0,0.00,2.00,30.00,2103389.0
sales_9_month,242075.0,511.775446,13976.702192,0.0,0.00,4.00,46.00,3195211.0


In [8]:
dataset.loc[:,'went_on_backorder'].value_counts()

No     239387
Yes      2688
Name: went_on_backorder, dtype: int64

In [9]:
dataset.drop('sku', axis=1, inplace=True)

In [10]:
dataset.isnull().sum()

national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          1
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                1
oe_constraint            1
ppap_risk                1
stop_auto_buy            1
rev_stop                 1
went_on_backorder        1
dtype: int64

In [11]:
yes_no_columns = list(filter(lambda i: dataset[i].dtype!=np.float64, dataset.columns))
print(yes_no_columns)

# Add code below this comment  (Question #E102)
# ----------------------------------
print("potential_issue:", dataset.potential_issue.unique())
print("deck_risk:", dataset.deck_risk.unique())
print("oe_constraint", dataset.oe_constraint.unique())
print("ppap_risk", dataset.ppap_risk.unique())
print("stop_auto_buy", dataset.stop_auto_buy.unique())
print("rev_stop", dataset.rev_stop.unique())
print('went_on_backorder', dataset['went_on_backorder'].unique())


['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']
potential_issue: ['No' 'Yes' nan]
deck_risk: ['No' 'Yes' nan]
oe_constraint ['No' 'Yes' nan]
ppap_risk ['No' 'Yes' nan]
stop_auto_buy ['Yes' 'No' nan]
rev_stop ['No' 'Yes' nan]
went_on_backorder ['No' 'Yes' nan]


In [12]:
for column_name in yes_no_columns:
    mode = dataset[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    dataset[column_name].fillna(mode, inplace=True)

Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No


In [13]:
dataset.isnull().sum()

national_inv             1
lead_time            14725
in_transit_qty           1
forecast_3_month         1
forecast_6_month         1
forecast_9_month         1
sales_1_month            1
sales_3_month            1
sales_6_month            1
sales_9_month            1
min_bank                 1
potential_issue          0
pieces_past_due          1
perf_6_month_avg         1
perf_12_month_avg        1
local_bo_qty             1
deck_risk                0
oe_constraint            0
ppap_risk                0
stop_auto_buy            0
rev_stop                 0
went_on_backorder        0
dtype: int64

In [14]:
dataset.loc[:,'lead_time'].isnull().value_counts()

False    227351
True      14725
Name: lead_time, dtype: int64

In [15]:
dataset['lead_time'].fillna(dataset['lead_time'].mode()[0], inplace=True)

dataset = dataset.dropna()
dataset.isnull().sum()

national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64

In [16]:
cat = dataset.select_dtypes(include = ['object']).columns
for col in cat:
    dataset[col].replace({'No': 0, 'Yes': 1}, inplace=True)
    dataset[col] = dataset[col].astype(int)

In [17]:
print("potential_issue:", dataset.potential_issue.unique())
print("deck_risk:", dataset.deck_risk.unique())
print("oe_constraint", dataset.oe_constraint.unique())
print("ppap_risk", dataset.ppap_risk.unique())
print("stop_auto_buy", dataset.stop_auto_buy.unique())
print("rev_stop", dataset.rev_stop.unique())
print('went_on_backorder', dataset['went_on_backorder'].unique())


potential_issue: [0 1]
deck_risk: [0 1]
oe_constraint [0 1]
ppap_risk [0 1]
stop_auto_buy [1 0]
rev_stop [0 1]
went_on_backorder [0 1]


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242075 entries, 0 to 242075
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   national_inv       242075 non-null  float64
 1   lead_time          242075 non-null  float64
 2   in_transit_qty     242075 non-null  float64
 3   forecast_3_month   242075 non-null  float64
 4   forecast_6_month   242075 non-null  float64
 5   forecast_9_month   242075 non-null  float64
 6   sales_1_month      242075 non-null  float64
 7   sales_3_month      242075 non-null  float64
 8   sales_6_month      242075 non-null  float64
 9   sales_9_month      242075 non-null  float64
 10  min_bank           242075 non-null  float64
 11  potential_issue    242075 non-null  int64  
 12  pieces_past_due    242075 non-null  float64
 13  perf_6_month_avg   242075 non-null  float64
 14  perf_12_month_avg  242075 non-null  float64
 15  local_bo_qty       242075 non-null  float64
 16  de

We can now predict and evaluate with the preprocessed test set. It would be interesting to see the performance with and without outliers removal from the test set. We can report confusion matrix, precision, recall, f1-score, accuracy, and other measures (if any). 

# Making X and y split of Testing dataset for final prediction

In [19]:
X_test = np.array(dataset.iloc[:,:-1]) # Pull all rows, each column except the last
y_test = np.array(dataset.went_on_backorder) # Pull just the went_on_backorder column
print('X_test', X_test.shape, 'y_test', y_test.shape)


X_test (242075, 21) y_test (242075,)


In [20]:
X_train_inliers, y_train_inliers, final_model = joblib.load('final_model.pkl')

In [21]:
#Model prediction with outliers
predicted_y = final_model.predict(X_test)

# classification report
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    239387
           1       0.08      0.80      0.15      2688

    accuracy                           0.90    242075
   macro avg       0.54      0.85      0.55    242075
weighted avg       0.99      0.90      0.94    242075



In [23]:
#confusion matrix
pd.DataFrame(confusion_matrix(y_test, predicted_y))

,0,1
0,215882,23505
1,525,2163


In [25]:
#Removing outliers from test dataset
anamoly_model.fit(X_test, y_test)
outliers_test = anamoly_model.predict(X_test)==-1
print(f"Num of outliers = {np.sum(outliers_test)}")
X_test_inliers = X_test[~outliers_test]
y_test_inliers = y_test[~outliers_test]

Num of outliers = 12104


In [26]:
#Model prediction without outliers
predicted_y_inliers = final_model.predict(X_test_inliers)

# classification report
print(classification_report(y_test_inliers, predicted_y_inliers))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    227414
           1       0.08      0.81      0.15      2557

    accuracy                           0.90    229971
   macro avg       0.54      0.85      0.55    229971
weighted avg       0.99      0.90      0.94    229971



In [27]:
#confusion matrix
pd.DataFrame(confusion_matrix(y_test_inliers, predicted_y_inliers))

,0,1
0,204595,22819
1,488,2069


## Conclusion

## Reflect

Imagine you are data scientist that has been tasked with developing a system to save your 
company money by predicting and preventing back orders of parts in the supply chain.

Write a **brief summary** for "management" that details your findings, 
your level of certainty and trust in the models, 
and recommendations for operationalizing these models for the business.

# Save your notebook!
## Then `File > Close and Halt`